In [1]:
import Model
import tensorflow as tf
import numpy as np
from extractMFCC import computeFeatures, computeFeatures1
from addNoise import addNoise

SyntaxError: invalid syntax (extractMFCC.py, line 69)